In [2]:
import boto3
import os


# 1. S3 select api introduction
When we use frameworks such as Spark, or Arrow to retrieve objects from s3. They always retrieve the whole entities of the objects. For example, if spark read a paruqet file of 10 GiB in s3, a total 10 GiB of data will be transfered from s3 to the spark cluster. Even though spark may just require some of the columns and rows to do the calculation. As a result, we retrived many useless data that increase the I/O of the operation.

To avoid this, the S3 Select API allows us to retrieve a subset of data by using simple SQL expressions. The data filtering happens on the s3 server. And only the data needed by the application will be retrieved. This can improve drasticly the operation performance. 

As a result, the S3 select api is designed for filtering columns and rows only. It's not designed for handling complex analytical queries and return results. 

# 2. Limitation of S3 Select
- It supports a maximum of 256 KB length of an SQL expression.
- It supports a maximum of 1 MB length of a record in the input or result.
- Few SQL clauses that are supported are **SELECT, FROM, WHERE, LIMIT**, etc.
- It is not useful for complex analytical queries and joins.
- Currently, only three object formats, **CSV, JSON, or Apache Parquet** are supported by S3 Select queries.
- At a time, the select query can execute on a single file (object).
- Minio only supports UTF-8 as the encoding type for select API.
- **AWS S3** supports compression such as: NONE, GZIP, BZIP2. The default Value is NONE. **Minio** supports GZIP, Snappy, LZ4 for columnar compression of Parquet API. Whole object compression is not supported for Parquet objects.

The first four limitations are normal, because S3 select is not designed for handling complex analytical queries.

The last four limitations are quite annoying. Because parquet file are often partitioned into multiple blocks. And parquet supports much more compression algo that s3 select can't read.

In [3]:
key_id=os.getenv("AWS_ACCESS_KEY_ID")
secret=os.getenv("AWS_SECRET_ACCESS_KEY")
session_token=os.getenv("AWS_SESSION_TOKEN")
endpoint=os.getenv("AWS_S3_ENDPOINT")

# print(f"key id: {key_id}")
# print(f"key secret: {secret}")
# print(f"session token: {session_token}")

In [4]:
s3_client = boto3.client(
    's3',
    endpoint_url=f'https://{endpoint}',
    aws_access_key_id=key_id,
    aws_secret_access_key=secret,
    aws_session_token=session_token
)

response = s3_client.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  donnees-insee
  pengfei
  projet-relevanc
  projet-spark-lab


In [59]:
# read csv file by using s3 select
def fetch_csv(bucket:str,path:str,query:str):
    """ Read csv file from s3 by using s3 select

    Keyword arguments:
    bucket -- the name of the bucket that the CSV file is in
    path -- the path to the csv file 
    query -- the sql query specification that will be used to filter the data.
    """
    response = s3_client.select_object_content(
               Bucket=bucket,
               Key=path,
               ExpressionType='SQL',
               Expression=query,
               InputSerialization = {'CSV': {"FileHeaderInfo": "Use"}, 'CompressionType': 'NONE'},
               OutputSerialization = {'CSV': {}},)
    return response


def print_csv_content(response):
    """ Print the content of the s3 select query
    
    Keyword arguments:
    response -- the response of the boto client s3 call 
    """
    
    for event in response['Payload']:
        if 'Records' in event:
            records = event['Records']['Payload'].decode('utf-8')
            print(records)
            
    
def print_query_stats(response):
    """ Print the detailed stats of the s3 select query 
    
    Keyword arguments:
    response -- the response of the boto client s3 call 
    """
    
    for event in response['Payload']:
        if 'Stats' in event:
            stats_details = event['Stats']['Details']
            print(f"Query bytes scanned : {stats_details['BytesScanned']} ")
            print(f"Query bytes processed: {stats_details['BytesProcessed']} ")
            print(f"Query bytes returned: {stats_details['BytesReturned']}")


In [38]:
# read csv with s3 select.
bucket="pengfei"
path="diffusion/data_format/netflix.csv"
q1="SELECT * FROM s3object s where s.\"rating\" = '5'"


In [23]:
resp1=fetch_csv(bucket,path,q1)        

print_query_stats(resp1)


Query bytes scanned : 495032025 
Query bytes processed: 495032025 
Query bytes returned: 113321890


In [61]:
# resp1=fetch_csv(bucket,path,q1)        
# print_csv_content(resp1)

In [62]:
q2="SELECT * FROM s3object"
resp2=fetch_csv(bucket,path,q2)

print_query_stats(resp2)
# print_csv_content(resp2)

Query bytes scanned : 495032025 
Query bytes processed: 495032025 
Query bytes returned: 495032005


You can notice the above two query scanned and processed the same amout of data. But in q1 we filter rows where "rating"='5', so it only returns 113321890B (113 MiB). In q2, we just select all the data, so it returns 495032005B (495 MiB).

## 3.2 Use s3 select to read parquet 

In minio, the **s3 select Parquet is DISABLED** by default since hostile crafted input can easily crash the server.

If you are in a controlled environment where it is safe to assume no hostile content can be uploaded to your cluster you can safely enable Parquet. To enable Parquet set the environment variable MINIO_API_SELECT_PARQUET=on.

You can find the official minio s3 select doc [here](https://docs.min.io/docs/minio-select-api-quickstart-guide.html)


In [63]:
# read parquet with s3 select.
data_path="diffusion/data_format/sf_fire/parquet/arrow_sf_fire_none/f402f99cb6d9459696314909b6f6e0a3.parquet"

resp = s3_client.select_object_content(
    Bucket='pengfei',
    Key=data_path,
    ExpressionType='SQL',
    Expression="SELECT * FROM s3object limit 5",
    InputSerialization = {'Parquet': {}, 'CompressionType': 'NONE'},
    OutputSerialization = {'CSV': {}},
)

ClientError: An error occurred (InternalError) when calling the SelectObjectContent operation (reached max retries: 4): We encountered an internal error, please try again.: cause(parquet format parsing not enabled on server)